Enable the `autoreload` extension, which will automatically reload the `playfab_manager` module when the source code is updated, using the following magic commands:

In [1]:
%load_ext autoreload
%autoreload 2

Import the `PlayFabManager` class from the `playfab_manager` module and create a `PlayFabManager` instance

In [3]:
from playfab_manager.playfab_manager import PlayFabManager
playfab = PlayFabManager()

Entity token is created


Use the `get_all_players` method of the `PlayFabManager` instance to retrieve a list of all players from the PlayFab:

In [4]:
# NOTE: players are already sorted by last login time
playfab.get_all_players()

Found 145 players


## Get traces

First, select recent players and get data for them

In [7]:
# Import the datetime module
import datetime

# Get the current time
current_time = datetime.datetime.now(datetime.timezone.utc)

# select the players in the last 24 hours
current_players = [p for p in playfab.all_players if current_time - p.LastLogin <= datetime.timedelta(hours=24)]

# print the last three player ids
[p.PlayerId for p in current_players[:3]]

['2F3BA89351BC480A', '689DD7AEB301A55E', 'CF40933BA34EDB9']

Second, get files for each of the player

In [9]:
playfab.download_player_files([p.PlayerId for p in current_players[:3]])

Found 2 files for player 2F3BA89351BC480A
Found 2 files for player 2F3BA89351BC480A
Found 2 files for player 689DD7AEB301A55E
Found 2 files for player 689DD7AEB301A55E
Found 2 files for player CF40933BA34EDB9
Found 2 files for player CF40933BA34EDB9


Merged files are stored in `Traces`

In [10]:
playfab.all_players[0].TracesPandas.head()

,timestamp,x,z,rotation,signaling
0,207019,0,-125,0,0
1,207119,0,-125,0,0
2,207219,0,-125,0,0
3,207319,0,-125,0,0
4,207419,0,-125,0,0
